In [ ]:
import pandas as pd
from pathlib import Path
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_info_columns', 10000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)


import seaborn as sns
%matplotlib inline
import re

In [ ]:
def cut_decimal_part(df, column):
    """
    Converts a DataFrame column from float (or numeric string) to a string
    by removing the decimal part (i.e. converting 13.5 to "13", 12.0 to "12").
    Non-numeric values are converted to NaN and then to an empty string.
    """
    df[column] = pd.to_numeric(df[column], errors='coerce')

    df[column] = df[column].apply(lambda x: str(int(x)) if pd.notnull(x) else '')
    
    return df

In [ ]:
def handle_null_values(df, fill_str="", fill_float=0.0, fill_datetime=""):
    """
    Fill null values in DataFrame columns based on data type.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        fill_str (str): Value to replace nulls in object/string columns. Default is "".
        fill_float (float): Value to replace nulls in float columns. Default is 0.0.
        fill_datetime: Value to replace nulls in datetime columns. 
                       Default is "", but you can also pass a default datetime.
    
    Returns:
        pd.DataFrame: The DataFrame with nulls handled.
    """

    obj_cols = df.select_dtypes(include=['object']).columns
    for col in obj_cols:
        df[col] = df[col].fillna(fill_str).astype(str)
    

    float_cols = df.select_dtypes(include=['float64']).columns
    for col in float_cols:
        df[col] = df[col].fillna(fill_float)
        

    datetime_cols = df.select_dtypes(include=['datetime64[ns]']) 
    for col in datetime_cols:
        df[col] = df[col].fillna(fill_datetime)
        
    return df

In [ ]:
import pandas as pd

def get_dataframe_summary(df):
    """
    Returns a summary DataFrame for the given DataFrame.
    
    The summary includes:
      - Data Type
      - Non Null Count
      - Null Count
      - Null Percentage
      - Unique Values count
    """
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    
    summary_df = pd.DataFrame({
        'Data Type': df.dtypes,
        'Non Null Count': df.count(),
        'Null Count': df.isna().sum(),
        'Null Percentage': (df.isna().sum() / len(df) * 100).round(2),
        'Unique Values': [df[col].nunique() for col in df.columns],
    })
    
    return summary_df

In [ ]:
BASE_DIR = Path.cwd().parent.parent.parent.parent.parent.parent.parent.parent
SAVE_DIR_EXTRACT_EXCEL = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "excel"/ "CORTE 3 _ 10-03-25 AL 16-03-25_20250328_182204.xlsx"
SAVE_DIR_EXTRACT_SGA_335 = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sga_335" / "sga_reporte_10-03-2025_16-03-2025_20250402_112253.xlsx"
CID_CUISMP_PATH = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sharepoint_cid_cuismp" / "MINPU - CID-CUISMP - AB.xlsx"

In [ ]:
df_corte_excel = pd.read_excel(SAVE_DIR_EXTRACT_EXCEL, skipfooter=2, engine="openpyxl")

In [ ]:
info_df_excel2 = get_dataframe_summary(df_corte_excel)
df_corte_excel = cut_decimal_part(df_corte_excel, 'CUISMP')

info_df_excel2

In [ ]:
df_corte_excel = handle_null_values(df_corte_excel)
info_df_excel2 = get_dataframe_summary(df_corte_excel)
info_df_excel2

In [ ]:
row = df_corte_excel[df_corte_excel['TICKET'] == 21786971 ]
row

In [ ]:
df_sga_dinamico_335 = pd.read_excel(SAVE_DIR_EXTRACT_SGA_335)
info_df_335 = get_dataframe_summary(df_sga_dinamico_335)
info_df_335
row = df_sga_dinamico_335[df_sga_dinamico_335['nro_incidencia'] == 21786971]
row

In [ ]:
df_sga_dinamico_335.head(1)

In [ ]:
info_sga_dinamico_335 = get_dataframe_summary(df_sga_dinamico_335)
info_sga_dinamico_335

In [ ]:
df_sga_dinamico_335['interrupcion_inicio'] = pd.to_datetime(df_sga_dinamico_335['interrupcion_inicio'], errors='coerce', dayfirst=True)
df_sga_dinamico_335['interrupcion_fin'] = pd.to_datetime(df_sga_dinamico_335['interrupcion_fin'], errors='coerce', dayfirst=True)
df_sga_dinamico_335['fecha_comunicacion_cliente'] = pd.to_datetime(df_sga_dinamico_335['fecha_comunicacion_cliente'], errors='coerce', dayfirst=True)
df_sga_dinamico_335['fecha_generacion'] = pd.to_datetime(df_sga_dinamico_335['fecha_generacion'], errors='coerce', dayfirst=True)
df_sga_dinamico_335['fg_padre'] = pd.to_datetime(df_sga_dinamico_335['fg_padre'], errors='coerce', dayfirst=True)
df_sga_dinamico_335['hora_sistema'] = pd.to_datetime(df_sga_dinamico_335['hora_sistema'], errors='coerce', dayfirst=True)

In [ ]:
import numpy as np
df_sga_dinamico_335['Expected_Inicio'] = np.where(df_sga_dinamico_335['masivo'] == "Si",
                                     df_sga_dinamico_335['fecha_generacion'],
                                     df_sga_dinamico_335['interrupcion_inicio'])
df_sga_dinamico_335.head(1)

In [ ]:
info_sga_dinamico_335 = get_dataframe_summary(df_sga_dinamico_335)
info_sga_dinamico_335

In [ ]:
def merge_sga_335_corte_excel(
    df_corte_excel: pd.DataFrame, 
    df_sga_dinamico_335: pd.DataFrame, 
) -> pd.DataFrame:
    """
    Common merge function for Objective 1.
    
    Merges:
      - corte-excel  with sga_dinamico_335 on 'nro_incidencia'

    Returns a merged DataFrame with common columns needed.
    """

    df_corte_excel = df_corte_excel.rename(columns={'TICKET':'nro_incidencia'})
   
    df_corte_excel['nro_incidencia'] = df_corte_excel['nro_incidencia'].astype(str)
    df_sga_dinamico_335['nro_incidencia'] = df_sga_dinamico_335['nro_incidencia'].astype(str)

    merged_sga335_excel = pd.merge(
        df_corte_excel,
        df_sga_dinamico_335,
        on='nro_incidencia',
        how='left',
        suffixes=('_corte_excel', '_sga_dinamico_335')
    )

    return merged_sga335_excel
df_merge_sga_335_corte_excel = merge_sga_335_corte_excel(df_corte_excel, df_sga_dinamico_335)
df_merge_sga_335_corte_excel.shape

In [ ]:
row = df_merge_sga_335_corte_excel[df_merge_sga_335_corte_excel['nro_incidencia'] == "21787031" ]
row

In [ ]:
def validation_fecha_inicio_fin(merged_df: pd.DataFrame) -> pd.DataFrame:
    """
    Valida las columnas de fechas en CORTE EXCEL y las compara con las del REPORTE DINÁMICO 335.
    
    Reglas:
      1) En CORTE EXCEL, “FECHA Y HORA INICIO” y “FECHA Y HORA FIN” no deben ser vacías.
      2) A partir del “nro_incidencia” en REPORTE DINÁMICO 335:
            - Si la columna “Masivo” tiene el valor “Si”, se utiliza “FECHA_GENERACION” como fecha de inicio.
            - En caso contrario, se utiliza “INTERRUPCION_INICIO”.
      3) Se compara:
            - La fecha esperada de inicio (según lo anterior) con “FECHA Y HORA INICIO” de CORTE EXCEL.
            - “INTERRUPCION_FIN” de REPORTE DINÁMICO 335 con “FECHA Y HORA FIN” de CORTE EXCEL.
      4) Ambas comparaciones deben coincidir para considerarse válidas.
    
    Devuelve un DataFrame con las siguientes columnas adicionales:
      - 'NotEmpty': Flag que indica que las fechas de CORTE EXCEL no son vacías.
      - 'Fecha_Inicio_match': Flag que indica si la fecha de inicio es correcta.
      - 'Fecha_Fin_match': Flag que indica si la fecha de fin es correcta.
      - 'Validation_OK': True si todas las condiciones se cumplen.
      - 'fail_count': Número de validaciones fallidas.
    """
    df = merged_df.copy()

    df['NotEmpty'] = df['FECHA Y HORA INICIO'].notna() & df['FECHA Y HORA FIN'].notna()

    df['Expected_Inicio'] = np.where(df['masivo'] == "Si",
                                     df['fecha_generacion'],
                                     df['interrupcion_inicio'])

    df['Fecha_Inicio_match'] = df['Expected_Inicio'] == df['FECHA Y HORA INICIO']

    df['Fecha_Fin_match'] = df['interrupcion_fin'] == df['FECHA Y HORA FIN']

    df['Validation_OK'] = df['NotEmpty'] & df['Fecha_Inicio_match'] & df['Fecha_Fin_match']

    df['fail_count'] = (~df['NotEmpty']).astype(int) + (~df['Fecha_Inicio_match']).astype(int) + (~df['Fecha_Fin_match']).astype(int)

    return df
df_validation_fecha_inicio_fin = validation_fecha_inicio_fin(df_merge_sga_335_corte_excel)
df_validation_fecha_inicio_fin.head(1)
row = df_validation_fecha_inicio_fin[df_validation_fecha_inicio_fin['nro_incidencia'] == '21787031']
row
